In [1]:
#mounting to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#importing libraries
import numpy as np
import os
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [3]:
#preprocessing fns
def bandpass_filter(x, fs=173.61, low=0.5, high=40, order=4):
    b, a = butter(order, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, x)

def sliding_windows(x, fs=173.61, win_s=2.0, overlap=0.5):
    win_samples = int(win_s*fs)
    step = int(win_samples*(1-overlap))
    windows = []
    for start in range(0, len(x)-win_samples+1, step):
        windows.append(x[start:start+win_samples])
    return np.array(windows)

def zscore_window(win):
    return (win - np.mean(win)) / (np.std(win) + 1e-8)

In [4]:
#pfen fn
def ordinal_patterns(seq, m=3, tau=1):
    n_vectors = len(seq) - (m-1)*tau
    patterns = np.zeros((n_vectors, m), dtype=int)
    for i in range(n_vectors):
        vec = seq[i:i + m*tau:tau]
        ranks = np.argsort(np.argsort(vec))
        patterns[i] = ranks
    return patterns

def phi_of_patterns(patterns, r, n):
    N = patterns.shape[0]
    if N <= 1:
        return 0.0
    sim_sum = 0.0
    count = 0
    for i in range(N):
        for j in range(i+1, N):
            d = np.max(np.abs(patterns[i]-patterns[j]))
            mu = np.exp(-(d**n)/r)
            sim_sum += mu
            count += 1
    return sim_sum / count if count>0 else 0.0

def pfen(window, m=3, tau=1, r=None, n=2):
    std = np.std(window)
    if r is None:
        r = 0.2*std + 1e-12
    Pm = ordinal_patterns(window, m, tau)
    Pm1 = ordinal_patterns(window, m+1, tau)
    phi_m = phi_of_patterns(Pm, r, n)
    phi_m1 = phi_of_patterns(Pm1, r, n)
    if phi_m<=0 or phi_m1<=0:
        return 0.0
    return -np.log(phi_m1/phi_m)

In [23]:
#loading dataset
dataset_path = '/content/drive/MyDrive/EEG_Project/datasets/FNOSZ/'
label_map = {'F': 0, 'N': 1, 'O': 2, 'S': 3, 'Z': 4}

In [6]:
def bandpass_filter(sig, low=0.5, high=40, fs=173.61):
    b, a = butter(4, [low/(fs/2), high/(fs/2)], btype='band')
    return filtfilt(b, a, sig)

In [7]:
#sliding window- makes it faster
def sliding_windows(x, fs=173.61, win_s=2.0, overlap=0.5):
    win_samples = int(win_s*fs)
    step = int(win_samples*(1-overlap))
    return [x[i:i+win_samples] for i in range(0, len(x)-win_samples+1, step)]

In [8]:
#pfen(fast)
def fast_pfen(signal, m=3, r=0.15):
    x = np.array(signal)
    N = len(x)
    if N < m + 1:
        return 0
    std_x = np.std(x)
    if std_x == 0:
        return 0
    r *= std_x
    X = np.array([x[i:N-m+i+1] for i in range(m)]).T
    diff = np.abs(X[:, None, :] - X[None, :, :])
    phi = np.exp(-(diff**2).sum(axis=2) / (r**2))
    np.fill_diagonal(phi, 0)
    # Handle cases where the argument to log is not positive
    log_arg = phi.sum() / (N**2 - N + 1e-8)
    if log_arg <= 0:
        return 0.0
    return np.log(log_arg)

In [27]:
#loading each fnosz as multichannel
import os # Import the os module
dataset_path = "/content/drive/MyDrive/EEG_Project/datasets/FNOSZ/"
classes = ['F', 'N', 'O', 'S', 'Z']

all_features = []
all_labels = []

# Each sample = one set of 5 channels (F,N,O,S,Z)
# Assume each folder has same number of files and same filenames
file_names = sorted(os.listdir(os.path.join(dataset_path, 'F')))
for fname in file_names:
    channels = []
    valid = True
    for cls in classes:
        fpath = os.path.join(dataset_path, cls, fname)
        if not os.path.exists(fpath):
            valid = False
            break
        sig = np.loadtxt(fpath)
        sig = bandpass_filter(sig)
        channels.append(sig)
    if not valid:
        continue

    # make same length
    min_len = min(len(ch) for ch in channels)
    channels = [ch[:min_len] for ch in channels]
    channels = np.array(channels)  # shape: (5, N)

    # windowing (2-sec windows)
    win_samples = int(2*173.61)
    step = int(win_samples//2)
    for start in range(0, min_len-win_samples+1, step):
        window_set = channels[:, start:start+win_samples]
        # Compute PFEn for each channel
        feats = [fast_pfen(ch) for ch in window_set]
        all_features.append(feats)

        # Label based on seizure class
        if 'S' in fname or 'Z' in fname:
            all_labels.append(1)  # seizure
        else:
            all_labels.append(0)  # non-seizure

all_features = np.array(all_features)
all_labels = np.array(all_labels)

In [28]:
import os
import numpy as np

dataset_path = '/content/drive/MyDrive/EEG_Project/datasets/FNOSZ'

data = []
labels = []

label_map = {'F': 0, 'N': 1, 'O': 2, 'S': 3, 'Z': 4}

for folder, lbl in label_map.items():
    folder_path = os.path.join(dataset_path, folder)
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            if file.lower().endswith('.txt'):  # <-- case-insensitive now
                file_path = os.path.join(folder_path, file)
                sig = np.loadtxt(file_path)
                if sig.size > 0:
                    data.append(sig)
                    labels.append(lbl)

data = np.array(data, dtype=object)
labels = np.array(labels)

print("Data loaded:", len(data))
print("Labels loaded:", len(labels))


Data loaded: 500
Labels loaded: 500


In [34]:
from scipy.signal import butter, filtfilt

def bandpass_filter(sig, low=0.5, high=50, fs=173.61, order=5):
    nyq = 0.5 * fs
    low_cut = low / nyq
    high_cut = high / nyq
    b, a = butter(order, [low_cut, high_cut], btype='band')
    filtered_sig = filtfilt(b, a, sig)
    return filtered_sig

# Apply bandpass filter to all signals
filtered_data = [bandpass_filter(sig) for sig in data]


In [35]:
def sliding_windows(sig, window_size=1024, step_size=512):
    windows = []
    for start in range(0, len(sig) - window_size + 1, step_size):
        windows.append(sig[start:start+window_size])
    return windows

# Example: create windows per signal
windowed_data = [sliding_windows(sig) for sig in filtered_data]


In [36]:
# Placeholder PFEn function (replace with real implementation)
def permutation_fuzzy_entropy(sig, m=3, n=2, delay=1):
    # Example: returns 1 PFEn value per signal
    return np.random.rand()  # Replace with your PFEn calculation

# Compute PFEn for each signal
pf_features = []
for windows in windowed_data:
    pf_values = [permutation_fuzzy_entropy(w) for w in windows]
    pf_features.append(np.mean(pf_values))  # average PFEn per signal

X = np.array(pf_features).reshape(-1, 1)
print("PFEn feature shape:", X.shape)  # should be (n_samples, 1)


PFEn feature shape: (500, 1)


In [37]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, random_state=42, stratify=labels
)


In [38]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

clf = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


In [39]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Accuracy: 0.18

Classification Report:
               precision    recall  f1-score   support

           0       0.21      0.15      0.18        20
           1       0.12      0.15      0.13        20
           2       0.17      0.15      0.16        20
           3       0.13      0.20      0.16        20
           4       0.42      0.25      0.31        20

    accuracy                           0.18       100
   macro avg       0.21      0.18      0.19       100
weighted avg       0.21      0.18      0.19       100


Confusion Matrix:
 [[3 5 5 6 1]
 [3 3 4 7 3]
 [3 6 3 7 1]
 [5 4 5 4 2]
 [0 7 1 7 5]]
